In [1]:
!nvidia-smi

Sun Jan 23 19:48:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers
!pip install simpletransformers
!pip install wandb

     |████████████████████████████████| 3.4 MB 32.3 MB/s 
     |████████████████████████████████| 895 kB 69.5 MB/s 
     |████████████████████████████████| 596 kB 72.5 MB/s 
     |████████████████████████████████| 3.3 MB 62.3 MB/s 
     |████████████████████████████████| 67 kB 7.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 248 kB 24.5 MB/s 
     |████████████████████████████████| 9.3 MB 68.8 MB/s 
     |████████████████████████████████| 311 kB 70.0 MB/s 
     |████████████████████████████████| 1.2 MB 67.9 MB/s 
     |████████████████████████████████| 1.7 MB 37.1 MB/s 
     |████████████████████████████████| 43 kB 1.3 MB/s 
     |████████████████████████████████| 97 kB 9.2 MB/s 
     |████████████████████████████████| 142 kB 76.1 MB/s 
     |████████████████████████████████| 180 kB 75.2 MB/s 
     |████████████████████████████████| 63 kB

In [3]:
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel
from sklearn.preprocessing import LabelEncoder

In [4]:
model = ClassificationModel('bert', 'recobo/chemical-bert-uncased', args = {
'evaluate_during_training' : True,
'evaluate_during_training_verbose' : True,
'evaluate_during_training_steps' : 82,
'use_cached_eval_features ':True,
'use_early_stopping': True,
'early_stopping_delta': 0.01,
'early_stopping_metric':"auroc",
'early_stopping_metric_minimize' : False,
'early_stopping_patience' : 10,
'max_seq_length' : 128,
'num_train_epochs' : 10,
'train_batch_size' : 32,
'learning_rate' : 4e-5,
'reprocess_input_data': True, 
'overwrite_output_dir': True,
'auto_weights': True
}, num_labels=2,)

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at recobo/chemical-bert-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at recobo/chemical-

Downloading:   0%|          | 0.00/223k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/339 [00:00<?, ?B/s]

In [5]:
data = pd.read_table('/content/B3DB_classification.tsv')
data = data[pd.notnull(data['IUPAC_name'])]
X = data['IUPAC_name']
target=data['BBB+/BBB-'].values.tolist()
label_encoder = LabelEncoder()
y_classification = label_encoder.fit_transform(target)

In [6]:
y_classification.shape

(6170,)

In [7]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, y_classification, test_size = 0.2, random_state = 42)
xtrain, xvalid, ytrain, yvalid = train_test_split(xtrain, ytrain, test_size = 0.16, random_state = 42)

In [8]:
Xtrain = {'text':xtrain, 'labels':ytrain}
Xvalid = {'text':xvalid, 'labels':yvalid}
Xtest = {'text':xtest, 'labels':ytest}
train_df = pd.DataFrame(Xtrain)
valid_df = pd.DataFrame(Xvalid)
test_df = pd.DataFrame(Xtest)

In [9]:
xtrain

3735                             phenyl 2-hydroxybenzoate
3079    4-amino-n-[[(2s)-1-ethylpyrrolidin-2-yl]methyl...
3669                             (2s)-oxane-2-carboxamide
3495    10-[(2s)-3-(dimethylamino)-2-methylpropyl]phen...
90      (e)-1-n'-[2-[[5-[(dimethylamino)methyl]furan-2...
                              ...                        
3444    2-methoxy-10-[2-[(2s)-1-methylpiperidin-2-yl]e...
3434    (3s,4s)-3-[(4-methoxyphenoxy)methyl]-1-methyl-...
2741    (6s)-3-(acetyloxymethyl)-8-oxo-7-[(2-phenylace...
3953    n-[[3-(2,4-dimethoxyphenyl)-1,2,4-oxadiazol-5-...
599                        1,1,1-trichloro-2-fluoroethane
Name: IUPAC_name, Length: 4146, dtype: object

In [10]:
import logging

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [11]:
out = model.train_model(train_df, eval_df=valid_df,args={'wandb_project': 'bbb_model_iupac'})

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/4146 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_2_2


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for training.


...

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.6413456121326176, 'tp': 187, 'tn': 479, 'fp': 43, 'fn': 81, 'auroc': 0.8932349745525247, 'auprc': 0.8050162708844621, 'eval_loss': 1.0162910638433513}


Running Epoch 9 of 10:   0%|          | 0/130 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/790 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2
INFO:simpletransformers.classification.classification_model:{'mcc': 0.6401487795970017, 'tp': 192, 'tn': 473, 'fp': 49, 'fn': 76, 'auroc': 0.8911191170583861, 'auprc': 0.8027096939097079, 'eval_loss': 1.0203176440313608}
INFO:simpletransformers.classification.classification_model: Patience of 10 steps reached
INFO:simpletransformers.classification.classification_model: Training terminated.
INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.


In [12]:

model = ClassificationModel(
    "bert", "/content/outputs/best_model",args={'wandb_project': 'bbb_model_iupac'}
)

In [13]:
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(test_df,acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1234 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/155 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


Training loss,█▇▄▇▄▄▅▂▃▃▃▄█▁▄▁▁▁▁▁▂▃▃▃
auprc,▅▇▅███▇█▇▅▆▄▆▅▃▄▂▂▁▃▃▃▃▃
auroc,▃▅▁▇▇▆▇█▆▆▅▆▆▅▆▇▅▅▃▄▅▅▅▄
eval_loss,▁▁▂▁▁▂▂▂▃▂▅▃▄▆▅▅▆▇█▇▇███
fn,█▅█▆▄▃▅▂▃▂▅▁▄▄▂▃▃▃▃▃▃▃▃▃
fp,▂▃▁▂▄▅▂▅▇█▂█▃▄▇▆▆▆▇▅▇▆▅▆
global_step,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
lr,▅██▇▇▇▆▆▆▅▅▅▅▄▄▄▃▃▃▂▂▂▁▁
mcc,▁▆▂▅▆▆▆█▄▅▅▇▇▆▆▆▅▅▅▆▅▆▆▆
tn,▇▆█▇▅▄▇▄▂▁▇▁▆▅▂▃▃▃▂▄▂▃▄▃
tp,▁▄▁▃▅▆▄▇▆▇▄█▅▅▇▆▆▆▆▆▆▆▆▆


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6557703296971319, 'tp': 296, 'tn': 748, 'fp': 50, 'fn': 140, 'auroc': 0.9183624198109954, 'auprc': 0.8798279252489825, 'acc': 0.8460291734197731, 'eval_loss': 0.43000362919222923}


{'mcc': 0.6557703296971319, 'tp': 296, 'tn': 748, 'fp': 50, 'fn': 140, 'auroc': 0.9183624198109954, 'auprc': 0.8798279252489825, 'acc': 0.8460291734197731, 'eval_loss': 0.43000362919222923}


In [14]:
import sklearn
result, model_outputs, wrong_predictions = model.eval_model(valid_df,acc=sklearn.metrics.accuracy_score)
print(result)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/790 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_2_2


Running Evaluation:   0%|          | 0/99 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Initializing WandB run for evaluation.


INFO:simpletransformers.classification.classification_model:{'mcc': 0.6492896554652278, 'tp': 176, 'tn': 493, 'fp': 29, 'fn': 92, 'auroc': 0.9080566992623091, 'auprc': 0.8600566109667729, 'acc': 0.8468354430379746, 'eval_loss': 0.4597768813672692}


{'mcc': 0.6492896554652278, 'tp': 176, 'tn': 493, 'fp': 29, 'fn': 92, 'auroc': 0.9080566992623091, 'auprc': 0.8600566109667729, 'acc': 0.8468354430379746, 'eval_loss': 0.4597768813672692}


In [15]:
!sudo apt-get install git-lfs
!git lfs install
!git config --global credential.helper store

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,622 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and directories c

In [16]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/token.
        (Deprecated, will be removed in v0.3.0) To login with username and password instead, interrupt with Ctrl+C.
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token


In [17]:
from transformers import AutoModel
from transformers import AutoTokenizer

model = AutoModel.from_pretrained("/content/outputs/best_model")
tokenizer = AutoTokenizer.from_pretrained("/content/outputs/best_model")

Some weights of the model checkpoint at /content/outputs/best_model were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
model.push_to_hub("BBB_prediction_finetuned_classification_iupac")
tokenizer.push_to_hub("BBB_prediction_finetuned_classification_iupac")

Cloning https://huggingface.co/Parsa/BBB_prediction_finetuned_classification_iupac into local empty directory.


Upload file pytorch_model.bin:   0%|          | 3.36k/419M [00:00<?, ?B/s]

To https://huggingface.co/Parsa/BBB_prediction_finetuned_classification_iupac
   74d3342..9d772ea  main -> main

   74d3342..9d772ea  main -> main

To https://huggingface.co/Parsa/BBB_prediction_finetuned_classification_iupac
   9d772ea..e808485  main -> main

   9d772ea..e808485  main -> main



'https://huggingface.co/Parsa/BBB_prediction_finetuned_classification_iupac/commit/e808485a6a982938fe19341e547cbfc09fa4d9ed'